In [ ]:
#选取实验数据（01）
import os
import random
import pandas as pd

# 指定文件夹路径
folder_path = r'E:\课题组\个人\论文\多系统风险传导最短路径\媒体情绪\媒体数据_res'

# 用于存储选取的数据
selected_data = []

# 遍历文件夹下的每个Excel文件
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        
        # 读取Excel文件
        df = pd.read_excel(file_path)
        
        # 随机选取500行数据
        if len(df) <= 500:
            selected_data.append(df)
        else:
            selected_data.append(df.sample(n=500))
        
# 将选取的数据合并
selected_data_df = pd.concat(selected_data, ignore_index=True)

# 将选取的数据保存到新的Excel文件中
selected_data_df.to_excel('emotion_selected_data1.xlsx', index=False)

In [ ]:
#按照月份进行关联规则挖掘（02-1）
import os
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 指定文件夹路径
folder_path = r'E:\课题组\个人\论文\多系统风险传导最短路径\媒体情绪\媒体数据_分月'

# 定义用于存储关联规则的空列表
all_rules = []

# 遍历文件夹下的所有文件
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)
        
        # 读取Excel文件
        df = pd.read_excel(file_path)
        
        # 处理含有 NaN 的行
        df = df.dropna()
        
        # 将每个单元格中的项集数据进行预处理
        transactions = []
        for index, row in df.iterrows():
            # 检查是否为字符串类型，如果不是，则转换为字符串类型
            if isinstance(row['res_all'], str):
                # 去除前后空白字符并替换特殊字符
                items = row['res_all'].strip().replace('\n', '').replace('\r', '')
            else:
                items = str(row['res_all']).strip().replace('\n', '').replace('\r', '')
            transactions.append(items.split(','))
        
        # 使用TransactionEncoder将项集转换成适合关联规则挖掘的数据格式
        te = TransactionEncoder()
        te_ary = te.fit(transactions).transform(transactions)
        transactions_df = pd.DataFrame(te_ary, columns=te.columns_)
        
        # 使用Apriori算法找出频繁项集
        frequent_itemsets = apriori(transactions_df, min_support=0.05, use_colnames=True)
        
        # 根据频繁项集生成关联规则
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
        
        # 将关联规则添加到总列表中
        all_rules.append(rules)
        
        # 将关联规则结果保存到新的Excel文件中
        result_file_name = f'result_{file_name}'
        rules.to_excel(result_file_name, index=False)

In [ ]:
#（03）
import os
import pandas as pd
from openpyxl import load_workbook

# 指定文件夹路径
folder_path = r'C:\Users\17122\jupytercode\shortest_risk\emotion_net\emotion_allyear'

# 遍历文件夹下的所有文件
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        excel_file = os.path.join(folder_path, filename)
        
        # 读取 Excel 文件
        df = pd.read_excel(excel_file, sheet_name=None)

        # 处理每个工作表
        for sheet_name, sheet_df in df.items():
            # 过滤数据并进行处理
            if 'antecedents' in sheet_df.columns and 'consequents' in sheet_df.columns:
                sheet_df = sheet_df[~sheet_df['antecedents'].str.contains(',') & ~sheet_df['consequents'].str.contains(',')]
                sheet_df['antecedents'] = sheet_df['antecedents'].str.extract(r"'(.*?)'")
                sheet_df['consequents'] = sheet_df['consequents'].str.extract(r"'(.*?)'")
                sheet_df = sheet_df[['antecedents', 'consequents', 'confidence']]

                # 将处理后的数据写入新的 sheet2
                with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a') as writer:
                    try:
                        # 尝试打开现有的 Excel 工作簿
                        writer.book = load_workbook(excel_file)
                        if 'sheet2' in writer.book.sheetnames:
                            idx = writer.book.sheetnames.index('sheet2')
                            writer.book.remove(writer.book.worksheets[idx])
                        sheet_df.to_excel(writer, sheet_name='sheet2', index=False)
                    except FileNotFoundError:
                        # 如果文件不存在，创建并写入
                        sheet_df.to_excel(writer, sheet_name='sheet2', index=False)


In [ ]:
#（04）
import os
import pandas as pd
from glob import glob

# 定义函数将Excel表转换成邻接矩阵
def excel_to_adjacency_matrix(file_path):
    # 读取 Excel 文件
    df = pd.read_excel(file_path, sheet_name='sheet2')

    # 获取节点列表
    nodes = sorted(set(df.iloc[:, 0]).union(df.iloc[:, 1]))

    # 创建邻接矩阵
    adj_matrix = pd.DataFrame(index=nodes, columns=nodes).fillna(0)

    # 填充邻接矩阵
    for index, row in df.iterrows():
        source = row[0]
        target = row[1]
        weight = row[2]
        adj_matrix.loc[source, target] = weight
        #adj_matrix.loc[target, source] = weight  # 如果是无向图，需将两个方向都填充

    # 获取文件名
    file_name = os.path.basename(file_path)

    # 将结果写入新的 Excel 文件
    adj_matrix.to_excel(os.path.join(r'C:\Users\17122\jupytercode\shortest_risk\emotion_net\emotion_adjmatrix', f'adjacency_matrix_{file_name}'), index_label='Node')

# 指定文件夹路径
folder_path = r'C:\Users\17122\jupytercode\shortest_risk\emotion_net\emotion_net_onmonth'

# 获取指定文件夹下所有的 Excel 文件
excel_files = glob(os.path.join(folder_path, '*.xlsx'))

# 遍历每个 Excel 文件并转换成邻接矩阵
for file in excel_files:
    excel_to_adjacency_matrix(file)
